In [1]:
#importing all required modules
import numpy as np
import scipy as sp
from math import *


#bokeh
from bokeh.plotting import Figure, output_file, output_notebook, show
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.layouts import column

In [2]:
# RMS value of voltage
u = 230 

#time vector
t = np.linspace(0,0.08, 100)

#frequency & angular frequency
f = 50
omega = 2 * pi * f

#Resitance
R = 5

#Inductance
L = 0.1
XL = 2*pi*f*L

#Phase angle
phi=atan(XL/R)

#closing angle [rad]
alpha = 0

#Phase A
#Current response
ia = [(sqrt(2)*u/(sqrt(R**2+XL**2))*(sin(omega*k+alpha-phi)-sin(alpha-phi)*exp(-R/L*k))) for k in t]

#DC component of the current
iadc = [(sqrt(2)*u/(sqrt(R**2+XL**2))*-sin(alpha-phi)*(exp(-R/L*k))) for k in t]

#AC steady state current
iau = [(sqrt(2)*u/(sqrt(R**2+XL**2))*sin(omega*k+alpha-phi)) for k in t]

#Plotting part

output_file("closing_angle-1.html")

source = ColumnDataSource(data=dict(t=t,ia=ia,iadc=iadc,iau=iau))

plot = Figure(plot_width=800, plot_height=400, title = "Influence of a voltage closing angle on the current in a R-L circuit")
plot.title.text_font_size = '12pt'
plot.xaxis.axis_label="Time [s]"
plot.xaxis.axis_label_text_font_size = '10pt'
plot.yaxis.axis_label="Current [A]"
plot.yaxis.axis_label_text_font_size = '10pt'
plot.line('t', 'ia', source=source, line_width=3, line_alpha=0.6, legend="Circuit current")
plot.line('t', 'iadc', source=source, line_width=2, line_alpha=0.6, color="orange", legend="DC component")
plot.line('t', 'iau', source=source, line_width=2, line_alpha=0.6, color="green", legend="AC component")

#callback part

callback1 = CustomJS(args=dict(source=source), code="""
        
    //Rewriting values for JavaScript code
    var R = 5;
    var L = 0.1;
    var f = 50;
    var XL = 2*Math.PI*f*L;
    var omega = 2*Math.PI*f;
    var phi = Math.atan(XL/R);
    var u_n = 230
    
    // get data source from Callback args
    var data = source.get('data');   
    var alpha = cb_obj.get('value');
    
    
    
    //Indicating which part of the source are certain values
    t = data['t'];
    ia = data['ia'];
    iadc = data['iadc'];
    iau = data['iau'];
   
    
    for (i=0; i < t.length; i++) {
        ia[i]= Math.sqrt(2)*u_n/(Math.sqrt(Math.pow(R,2)+ Math.pow(XL,2)))*(Math.sin(omega*t[i]+alpha-phi)-Math.sin(alpha-phi)*Math.exp(-R/L*t[i]));
        iadc[i]= Math.sqrt(2)*u_n/(Math.sqrt(Math.pow(R,2)+ Math.pow(XL,2)))*-Math.sin(alpha-phi)*Math.exp(-R/L*t[i]);
        iau[i]= Math.sqrt(2)*u_n/(Math.sqrt(Math.pow(R,2)+ Math.pow(XL,2)))*(Math.sin(omega*t[i]+alpha-phi));
    };    
    
    source.trigger('change');

    """)




slider2 = Slider(start=0, end=phi, value=0, step=.1, title="Voltage closing angle [rad]", callback=callback1, name="slider2")

#inputs = VBoxForm(children=[slider2])

#layout = vform(inputs, plot)

layout = column(slider2, plot)

show(layout)
